# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
orders= pd.read_csv('Orders\Orders.csv')
orders.head(2)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# your code here
agg_amount_spent=orders.groupby(['CustomerID']).agg({'amount_spent':'sum'}).reset_index()
agg_amount_spent

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [20]:
agg_amount_spent['qbin']=pd.qcut(agg_amount_spent["amount_spent"],20,['0%','5%','10%','15%','20%','25%','30%','35%','40%','45%','50%','55%','60%','65%',
'70%','75%','80%','85%','90%','95%'])
#agg_amount_spent['qbin']=agg_amount_spent['qbin'].apply(pd.to_numeric)  (i dont know why it is not working)

In [23]:
vip_segment=agg_amount_spent[agg_amount_spent['qbin']=='95%']
vip_segment['CustomerID'] = vip_segment.index
vip_segment



C:\Users\Bachir\AppData\Local\Temp\ipykernel_16736\3222478222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vip_segment['CustomerID'] = vip_segment.index


,CustomerID,amount_spent,qbin
0,0,77183.60,95%
10,10,6207.67,95%
12,12,6372.58,95%
50,50,11072.67,95%
55,55,124914.53,95%
...,...,...,...
4207,4207,8052.97,95%
4229,4229,8438.34,95%
4253,4253,7561.68,95%
4292,4292,6484.54,95%


In [34]:
preferred_segment = agg_amount_spent[(agg_amount_spent['qbin'] == '75%') | (agg_amount_spent['qbin'] == '80%') | (agg_amount_spent['qbin'] == '85%')]
preferred_segment['CustomerID']=preferred_segment.index
preferred_segment

C:\Users\Bachir\AppData\Local\Temp\ipykernel_16736\1421797486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preferred_segment['CustomerID']=preferred_segment.index


,CustomerID,amount_spent,qbin
2,2,1797.24,75%
3,3,1757.55,75%
5,5,2506.04,80%
9,9,2811.43,85%
13,13,2662.06,85%
...,...,...,...
4319,4319,2338.60,80%
4320,4320,2643.20,85%
4328,4328,3078.58,85%
4337,4337,2094.88,80%


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [32]:
# your code here
vip_per_country = pd.merge(
                        vip_segment, 
                        orders, 
                        how = 'inner', 
                        left_index=True, 
                        right_on='CustomerID'
                        ) 
vip_per_country['qbin'].unique()
vip_per_country['Country'].max()

nan

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [35]:
# your code here
#group by the new extended table by country and count the lable column

vip_pref = pd.merge(vip_segment, preferred_segment, how = 'outer', left_index= True, right_index =True) 
vip_pref

,CustomerID_x,amount_spent_x,qbin_x,CustomerID_y,amount_spent_y,qbin_y
0,0.0,77183.6,95%,NaN,NaN,NaN
2,NaN,NaN,NaN,2.0,1797.24,75%
3,NaN,NaN,NaN,3.0,1757.55,75%
5,NaN,NaN,NaN,5.0,2506.04,80%
9,NaN,NaN,NaN,9.0,2811.43,85%
...,...,...,...,...,...,...
4319,NaN,NaN,NaN,4319.0,2338.60,80%
4320,NaN,NaN,NaN,4320.0,2643.20,85%
4328,NaN,NaN,NaN,4328.0,3078.58,85%
4337,NaN,NaN,NaN,4337.0,2094.88,80%
